# 네이버 매장 -> 매장에 있는 리뷰 스크래핑

문제
 
1. 추천순으로 긁어올지, 최신순으로 긁어올지

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException

import datetime
import time
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import requests
from openpyxl import Workbook
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
res_df = pd.read_excel('combined_file1_22.xlsx')
res_df = res_df.loc[res_df['검색여부']=='가능',:]
res_df.reset_index(inplace=True)
res_df.drop(['index','Unnamed: 0'],axis=1,inplace=True )

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys
import time
driver = webdriver.Chrome()

name = '강남대로 378, 채선당 샤브보트'
url='https://map.naver.com/p/search/{}'.format(name)


driver.get(url)
entryIframe = driver.find_element(By.ID,'entryIframe')
driver.switch_to.frame(entryIframe)
actions = driver.find_element(By.CSS_SELECTOR, 'body')

time.sleep(2)
actions.send_keys(Keys.END)
time.sleep(2)
actions.send_keys(Keys.END)
time.sleep(2)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="entryIframe"]"}
  (Session info: chrome=121.0.6167.185); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF701587012+3522402]
	(No symbol) [0x00007FF7011A8352]
	(No symbol) [0x00007FF701055ABB]
	(No symbol) [0x00007FF70109BF0E]
	(No symbol) [0x00007FF70109C08C]
	(No symbol) [0x00007FF7010DE437]
	(No symbol) [0x00007FF7010BF09F]
	(No symbol) [0x00007FF7010DBDA3]
	(No symbol) [0x00007FF7010BEE03]
	(No symbol) [0x00007FF70108F4D4]
	(No symbol) [0x00007FF7010905F1]
	GetHandleVerifier [0x00007FF7015B9B9D+3730157]
	GetHandleVerifier [0x00007FF70160F02D+4079485]
	GetHandleVerifier [0x00007FF7016075D3+4048163]
	GetHandleVerifier [0x00007FF7012DA649+718233]
	(No symbol) [0x00007FF7011B4A3F]
	(No symbol) [0x00007FF7011AFA94]
	(No symbol) [0x00007

In [3]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
# 주소 이동
url = 'https://map.naver.com/'
driver.get(url)
time.sleep(1)

In [7]:
# 페이지 스크롤 횟수만큼 반복
page_down = 10
name = '강남대로 378, 채선당 샤브보트'
driver.get('https://map.naver.com/p/search/{}'.format(name))
for _ in range(page_down):
    # 대기시간 5초 
    wait = WebDriverWait(driver, 5)

    # 요소를 찾을 때 대기 조건 추가
    
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN) 
    user_review_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'iqjjt')))


TimeoutException: Message: 


식당 검색하고 entryIframe 에서 스크롤 내리고 클릭 방문자 리뷰에서  클릭해야함


In [8]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver import ActionChains
# 여기서부터는 드라이버 설정 등의 코드가 와야합니다.


name = '강남대로 378, 채선당 샤브보트'
driver.get('https://map.naver.com/p/search/{}'.format(name))

time.sleep(3) 
find = driver.find_element(By.CLASS_NAME,'iqjjt')
action = ActionChains(driver)
action.move_to_element(find).perform()
time.sleep(3) 


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".iqjjt"}
  (Session info: chrome=121.0.6167.185); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF701587012+3522402]
	(No symbol) [0x00007FF7011A8352]
	(No symbol) [0x00007FF701055ABB]
	(No symbol) [0x00007FF70109BF0E]
	(No symbol) [0x00007FF70109C08C]
	(No symbol) [0x00007FF7010DE437]
	(No symbol) [0x00007FF7010BF09F]
	(No symbol) [0x00007FF7010DBDA3]
	(No symbol) [0x00007FF7010BEE03]
	(No symbol) [0x00007FF70108F4D4]
	(No symbol) [0x00007FF7010905F1]
	GetHandleVerifier [0x00007FF7015B9B9D+3730157]
	GetHandleVerifier [0x00007FF70160F02D+4079485]
	GetHandleVerifier [0x00007FF7016075D3+4048163]
	GetHandleVerifier [0x00007FF7012DA649+718233]
	(No symbol) [0x00007FF7011B4A3F]
	(No symbol) [0x00007FF7011AFA94]
	(No symbol) [0x00007FF7011AFBC2]

In [9]:
for i in range(0,1) :
    # 검색어 = 지역명 + 가게명 ex) 강남구 떡도리탕
    # name = res_df['상세주소'][i] + ' ' + res_df['사업장명'][i]
    name = '강남대로 378, 채선당 샤브보트'
    driver.get('https://map.naver.com/p/search/{}'.format(name))
    ## 화면이 안넘어가고 검색결과에 머물러 있을 때는 entryIframe 이 없음
    ## 그래서 검색어 쳤을때 entryIframe 있으면 --> 화면이 넘어갔다는 뜻이고 
    ## 없으면 아직 안넘어간 뜻이니깐 searchIframe 에서 클릭해서 넘어가야함 .
    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
            time.sleep(3) 
            # 스크롤 내려야함
            body = driver.find_element(By.ID,'iqjjt')
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(3) 
            # ## 태그 클릭 iqjjt
            # driver.find_element(By.CLASS_NAME,'place_section_content').click()
            # time.sleep(3) 
    except Exception as e :
        print('안됌')
            

In [19]:
for i in range(0,3) :
    # 검색어 = 지역명 + 가게명 ex) 강남구 떡도리탕
    # name = res_df['상세주소'][i] + ' ' + res_df['사업장명'][i]
    name = '강남대로114길 18, 정식당'

    ## 화면이 안넘어가고 검색결과에 머물러 있을 때는 entryIframe 이 없음
    ## 그래서 검색어 쳤을때 entryIframe 있으면 --> 화면이 넘어갔다는 뜻이고 
    ## 없으면 아직 안넘어간 뜻이니깐 searchIframe 에서 클릭해서 넘어가야함 .
    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
            
            time.sleep(3) 

            ## 스크롤 내리고 
            driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
            time.sleep(3) 

            ## 태그 클릭
            time.sleep(3) 
            driver.find_element(By.CLASS_NAME,'uNsI9').click()

            
            
        else : 
            # searchIframe 프레임 

                searchIframe = driver.find_element(By.ID,'searchIframe')
                driver.switch_to.frame(searchIframe)


                ## 클릭 
                driver.find_element(By.CLASS_NAME,'YwYLL').click()
                
                # 기본프렝미 
                time.sleep(1)
                driver.switch_to.default_content()
            

                # entryIframe 프레임 
                entryIframe = driver.find_element(By.ID,'entryIframe')
                driver.switch_to.frame(entryIframe)

                time.sleep(3) 
                # 가게 이름, 변수로 지정
                restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
                category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
                address = driver.find_element(By.CLASS_NAME,'LDgIH').text
                if driver.find_elements(By.CLASS_NAME,'ihmWt') :
                    menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
                    price = driver.find_element(By.CLASS_NAME,'mkBm3').text
                elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
                    menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
                    price = driver.find_element(By.CLASS_NAME,'Yrsei').text
                else : 
                    menu = '메뉴없음'
                    price = "가격없음"
    except Exception as e : 
        restaurant_name = name 
        category_name = '전처리필요'
        address = '전처리필요'
        menu = '메뉴없음'
        price = "가격없음"
        print(f"에러 메시지: {str(e)}")


에러 메시지: 'WebDriver' object has no attribute 'find_element_by_tag_name'
에러 메시지: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="searchIframe"]"}
  (Session info: chrome=121.0.6167.185); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF739EE7012+3522402]
	(No symbol) [0x00007FF739B08352]
	(No symbol) [0x00007FF7399B5ABB]
	(No symbol) [0x00007FF7399FBF0E]
	(No symbol) [0x00007FF7399FC08C]
	(No symbol) [0x00007FF739A3E437]
	(No symbol) [0x00007FF739A1F09F]
	(No symbol) [0x00007FF739A3BDA3]
	(No symbol) [0x00007FF739A1EE03]
	(No symbol) [0x00007FF7399EF4D4]
	(No symbol) [0x00007FF7399F05F1]
	GetHandleVerifier [0x00007FF739F19B9D+3730157]
	GetHandleVerifier [0x00007FF739F6F02D+4079485]
	GetHandleVerifier [0x00007FF739F675D3+4048163]
	GetHandleVerifier [0x00007FF739C3A649+718233]
	(No symbol) [0x00007FF739B14A3F]

In [71]:
# tag - 이런점이 좋았어요 가져오기 
# tag 끝까지 나오게 클릭 
try :
    while driver.find_elements(By.CLASS_NAME, 'Tvx37'):
        try:
            button = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[1]/div/div/div[2]/a[1]')
            button.click()
            time.sleep(0.4)
        except ElementClickInterceptedException:
            break
except NoSuchElementException:
    print("No more '더보기' button found. Exiting the loop.")

# tag 불러오기 
html = driver.page_source
soup = BeautifulSoup(driver.page_source, 'html.parser')
items = soup.select('ul.uNsI9 li.nbD78, ul.uNsI9 li.nbD78.Xcpsu')

# 'tag'와 '인원'을 추출하여 새로운 데이터프레임 생성
data = {
    'tag': [item.text.split('"')[1] for item in items],
    '인원': [int(''.join(filter(str.isdigit, item.text))) for item in items]
}

df = pd.DataFrame(data)

# 결과 출력
df

,tag,인원
0,음식이 맛있어요,1103
1,재료가 신선해요,723
2,혼밥하기 좋아요,612
3,친절해요,300
4,가성비가 좋아요,254
5,매장이 청결해요,241
6,매장이 넓어요,206
7,특별한 메뉴가 있어요,127
8,양이 많아요,122
9,인테리어가 멋져요,42


In [ ]:
url = 'https://m.place.naver.com/restaurant/13149660/review/visitor?entry=pll'

driver = webdriver.Chrome(ChromeDriverManager().install())
res = driver.get(url)
driver.implicitly_wait(10)

# # 최신순 리뷰
# driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').click()
# time.sleep(0.4)

# 추천순 리뷰
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/div[2]/div[1]/span[1]/a').click()
time.sleep(0.4)


# html = driver.page_source
# #soup = BeautifulSoup(html, 'lxml')
# soup = BeautifulSoup(driver.page_source, 'html.parser')


In [72]:
# 매장명
place = '강남역_서브웨이'

# 엑셀파일 만들기 
now = datetime.datetime.now()
xlsx = Workbook()
list_sheet = xlsx.create_sheet(place)
list_sheet.append(['nickname', 'content', 'date', 'revisit','tag'])

# 리뷰 스크래핑 시작
try:
    # driver = webdriver.Chrome(ChromeDriverManager().install())
    # res = driver.get(url)
    # driver.implicitly_wait(2)

    for _ in range(5): # 리뷰 더보기 5번만...
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(0.4)

        # Pagedown
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)

        try:
            while True:
                driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').click()
                time.sleep(0.4)
        except Exception as e:
            print('finish')

        time.sleep(30)
        html = driver.page_source
        bs = BeautifulSoup(html, 'lxml')
        reviews = bs.select('li.YeINN')

        for r in reviews:
            nickname = r.select_one('div.VYGLG')
            content = r.select_one('div.ZZ4OK.IwhtZ')
            tag = r.select_one('div.gyAGI span.P1zUJ').text
            date = r.select('div._7kR3e>span.tzZTd>time')[0]
            revisit = r.select('div._7kR3e>span.tzZTd')[1]

            # 아무것도 없을 때
            nickname = nickname.text if nickname else ''
            content = content.text if content else ''
            tag = tag if tag else ''
            date = date.text if date else ''
            revisit = revisit.text if revisit else ''
            time.sleep(0.06)

            print(nickname, '/', content, '/', date, '/', revisit, '/',tag)
            list_sheet.append([nickname, content, date, revisit,tag])
            time.sleep(0.06)

        # 저장하기
        file_name = 'naver_review_' + place + now.strftime('%Y-%m-%d_%H-%M-%S') + '.xlsx'
        xlsx.save(file_name)

# 에러가 나는 경우
except Exception as e:
    print(e)
    # Save the file(temp)
    file_name = 'naver_review_'+ place + now.strftime('%Y-%m-%d_%H-%M-%S') + '.xlsx'
    xlsx.save(file_name)

finish
맛집학과 수석 / 신메뉴 랍스터 나왔다해서 먹으러 왔어요!! 가격이 좀 비싸긴 했지만 랍스터 첫입 먹을 때 그 랍스터 향기는 잊을 수가 없네요!! 진짜 맛있구 또 먹을거에요!! :) / 23.12.11.월 / 8번째 방문 / 음식이 맛있어요
Favor10 / 키오스크가 있어서 편해요!! / 2.4.일 / 5번째 방문 / 음식이 맛있어요
토랑꾸 / 사람이 많은데 빨리 먹기 괜찮고, 맛있어요!
행사해서 다른 서브웨이보다 가격도 저렴하고 좋아요😀 / 1.22.월 / 1번째 방문 / 가성비가 좋아요
NancyK /  / 23.12.14.목 / 2번째 방문 / 음식이 맛있어요
쪼꼬미쪼콩 / 빠르고 맛있게 만들어줘요!👍 / 1.22.월 / 2번째 방문 / 음식이 맛있어요
별땅20 / 랍스터가 많이 들어있네요. 가격이 많이 비싸지만 맛있게 먹었네요.  / 23.12.26.화 / 1번째 방문 / 음식이 맛있어요
맛집학과 수석 / 진짜 여기 섭웨가 잘 만드시는 것 같아요! 로스팅이 항상 잘 되어나와서 바삭한 빵을 만날 수 있어요 :) / 23.11.30.목 / 7번째 방문 / 혼밥하기 좋아요
dldkdud2150 / 첫 서브웨이 
가벼운 아침메뉴 먹으러 갔고 매장도 쾌적하고 재료들도 신선해서 맛있었어요
쿠키도 쫀득하고 시켰던 메뉴들과 조합이 생각보다 좋더라구요 / 23.11.26.일 / 1번째 방문 / 매장이 넓어요
Favor10 / 햄버거땡길때..전 써브웨이로! 키오스크 밀리지않으면 속도도 빠르고 편해용! / 23.12.13.수 / 3번째 방문 / 음식이 맛있어요
맛집학과 수석 / 섭웨는 매일 먹어도 안질려요 :) / 1.26.금 / 12번째 방문 / 매장이 넓어요
맛집학과 수석 / 간편하게 혼밥하기 더할나위 없이 좋아요 :) / 23.11.9.목 / 5번째 방문 / 음식이 맛있어요
sae**** / 주문 방식이 키오스크여서 당황했어요 
그래서 써브웨이의 특유한 분위기가 사라진 것ㅁ같아 아쉽네요.  / 1.29.월 / 1번째 방문 / 음식이 맛있어요
쪼꼬미쪼콩 